In [1]:
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots

import matplotlib.pyplot as plt
import plotly.graph_objects as go

class CandlestickPatternRecognizer:
    def __init__(self, data):
        """
        Initialize with OHLC data
        data should be a DataFrame with columns: Open, High, Low, Close
        """
        self.data = data.copy()
        self.data.columns = ['Open', 'High', 'Low', 'Close']
        self._calculate_helpers()
    
    def _calculate_helpers(self):
        """Calculate helper columns for pattern recognition"""
        self.data['Body'] = abs(self.data['Close'] - self.data['Open'])
        self.data['UpperShadow'] = self.data['High'] - self.data[['Open', 'Close']].max(axis=1)
        self.data['LowerShadow'] = self.data[['Open', 'Close']].min(axis=1) - self.data['Low']
        self.data['IsGreen'] = self.data['Close'] > self.data['Open']
        self.data['IsRed'] = self.data['Close'] < self.data['Open']
        self.data['IsDoji'] = self.data['Body'] < (self.data['High'] - self.data['Low']) * 0.1
        
        # Additional helper calculations
        self.data['Range'] = self.data['High'] - self.data['Low']
        self.data['BodyMidpoint'] = (self.data['Open'] + self.data['Close']) / 2
        self.data['IsMarubozu'] = (self.data['UpperShadow'] < self.data['Range'] * 0.05) & (self.data['LowerShadow'] < self.data['Range'] * 0.05)
    
    def detect_hammer(self, threshold=2):
        """Detect Hammer pattern"""
        conditions = (
            (self.data['LowerShadow'] >= threshold * self.data['Body']) &
            (self.data['UpperShadow'] <= 0.1 * self.data['Body']) &
            (self.data['Body'] > 0)
        )
        return conditions
    
    def detect_hanging_man(self, threshold=2):
        """Detect Hanging Man pattern (same as hammer but in uptrend)"""
        return self.detect_hammer(threshold)
    
    def detect_doji(self):
        """Detect Doji pattern"""
        return self.data['IsDoji']
    
    def detect_long_legged_doji(self):
        """Detect Long-Legged Doji pattern"""
        long_shadows = (self.data['UpperShadow'] + self.data['LowerShadow']) > self.data['Body'] * 5
        return self.data['IsDoji'] & long_shadows
    
    def detect_high_wave_candle(self):
        """Detect High-Wave Candle pattern"""
        long_shadows = (self.data['UpperShadow'] + self.data['LowerShadow']) > self.data['Body'] * 3
        small_body = self.data['Body'] < self.data['Range'] * 0.3
        return long_shadows & small_body
    
    def detect_engulfing_bullish(self):
        """Detect Bullish Engulfing pattern"""
        if len(self.data) < 2:
            return pd.Series([False] * len(self.data))
        
        prev_red = self.data['IsRed'].shift(1)
        curr_green = self.data['IsGreen']
        curr_open_below_prev_close = self.data['Open'] < self.data['Close'].shift(1)
        curr_close_above_prev_open = self.data['Close'] > self.data['Open'].shift(1)
        
        return (prev_red & curr_green & curr_open_below_prev_close & curr_close_above_prev_open).fillna(False)
    
    def detect_engulfing_bearish(self):
        """Detect Bearish Engulfing pattern"""
        if len(self.data) < 2:
            return pd.Series([False] * len(self.data))
        
        prev_green = self.data['IsGreen'].shift(1)
        curr_red = self.data['IsRed']
        curr_open_above_prev_close = self.data['Open'] > self.data['Close'].shift(1)
        curr_close_below_prev_open = self.data['Close'] < self.data['Open'].shift(1)
        
        return (prev_green & curr_red & curr_open_above_prev_close & curr_close_below_prev_open).fillna(False)
    
    def detect_harami_bullish(self):
        """Detect Bullish Harami pattern"""
        if len(self.data) < 2:
            return pd.Series([False] * len(self.data))
        
        prev_red = self.data['IsRed'].shift(1)
        prev_large_body = self.data['Body'].shift(1) > self.data['Body'].shift(1).rolling(10).mean()
        curr_small_body = self.data['Body'] < self.data['Body'].shift(1) * 0.5
        curr_contained = (self.data['Open'] > self.data['Close'].shift(1)) & (self.data['Close'] < self.data['Open'].shift(1))
        
        return (prev_red & prev_large_body & curr_small_body & curr_contained).fillna(False)
    
    def detect_harami_bearish(self):
        """Detect Bearish Harami pattern"""
        if len(self.data) < 2:
            return pd.Series([False] * len(self.data))
        
        prev_green = self.data['IsGreen'].shift(1)
        prev_large_body = self.data['Body'].shift(1) > self.data['Body'].shift(1).rolling(10).mean()
        curr_small_body = self.data['Body'] < self.data['Body'].shift(1) * 0.5
        curr_contained = (self.data['Open'] < self.data['Close'].shift(1)) & (self.data['Close'] > self.data['Open'].shift(1))
        
        return (prev_green & prev_large_body & curr_small_body & curr_contained).fillna(False)
    
    def detect_harami_cross_bullish(self):
        """Detect Bullish Harami Cross pattern"""
        if len(self.data) < 2:
            return pd.Series([False] * len(self.data))
        
        prev_red = self.data['IsRed'].shift(1)
        prev_large_body = self.data['Body'].shift(1) > self.data['Body'].shift(1).rolling(10).mean()
        curr_doji = self.data['IsDoji']
        curr_contained = (self.data['BodyMidpoint'] > self.data['Close'].shift(1)) & (self.data['BodyMidpoint'] < self.data['Open'].shift(1))
        
        return (prev_red & prev_large_body & curr_doji & curr_contained).fillna(False)
    
    def detect_harami_cross_bearish(self):
        """Detect Bearish Harami Cross pattern"""
        if len(self.data) < 2:
            return pd.Series([False] * len(self.data))
        
        prev_green = self.data['IsGreen'].shift(1)
        prev_large_body = self.data['Body'].shift(1) > self.data['Body'].shift(1).rolling(10).mean()
        curr_doji = self.data['IsDoji']
        curr_contained = (self.data['BodyMidpoint'] < self.data['Close'].shift(1)) & (self.data['BodyMidpoint'] > self.data['Open'].shift(1))
        
        return (prev_green & prev_large_body & curr_doji & curr_contained).fillna(False)
    
    def detect_three_inside_up(self):
        """Detect Three Inside Up pattern"""
        if len(self.data) < 3:
            return pd.Series([False] * len(self.data))
        
        # First candle: Large red
        first_red = self.data['IsRed'].shift(2)
        first_large = self.data['Body'].shift(2) > self.data['Body'].shift(2).rolling(10).mean()
        
        # Second candle: Small body contained within first candle (Harami)
        second_contained = (
            (self.data['Open'].shift(1) > self.data['Close'].shift(2)) &
            (self.data['Close'].shift(1) < self.data['Open'].shift(2))
        )
        second_small = self.data['Body'].shift(1) < self.data['Body'].shift(2) * 0.5
        
        # Third candle: Green closing above first candle's midpoint
        third_green = self.data['IsGreen']
        third_closes_high = self.data['Close'] > (self.data['Open'].shift(2) + self.data['Close'].shift(2)) / 2
        
        return (first_red & first_large & second_contained & second_small & third_green & third_closes_high).fillna(False)
    
    def detect_three_inside_down(self):
        """Detect Three Inside Down pattern"""
        if len(self.data) < 3:
            return pd.Series([False] * len(self.data))
        
        # First candle: Large green
        first_green = self.data['IsGreen'].shift(2)
        first_large = self.data['Body'].shift(2) > self.data['Body'].shift(2).rolling(10).mean()
        
        # Second candle: Small body contained within first candle (Harami)
        second_contained = (
            (self.data['Open'].shift(1) < self.data['Close'].shift(2)) &
            (self.data['Close'].shift(1) > self.data['Open'].shift(2))
        )
        second_small = self.data['Body'].shift(1) < self.data['Body'].shift(2) * 0.5
        
        # Third candle: Red closing below first candle's midpoint
        third_red = self.data['IsRed']
        third_closes_low = self.data['Close'] < (self.data['Open'].shift(2) + self.data['Close'].shift(2)) / 2
        
        return (first_green & first_large & second_contained & second_small & third_red & third_closes_low).fillna(False)
    
    def detect_marubozu_bullish(self):
        """Detect Bullish Marubozu pattern"""
        return self.data['IsGreen'] & self.data['IsMarubozu']
    
    def detect_marubozu_bearish(self):
        """Detect Bearish Marubozu pattern"""
        return self.data['IsRed'] & self.data['IsMarubozu']
    
    def detect_kicking_bullish(self):
        """Detect Bullish Kicking pattern"""
        if len(self.data) < 2:
            return pd.Series([False] * len(self.data))
        
        prev_bearish_marubozu = self.data['IsRed'].shift(1) & self.data['IsMarubozu'].shift(1)
        curr_bullish_marubozu = self.data['IsGreen'] & self.data['IsMarubozu']
        gap_up = self.data['Open'] > self.data['High'].shift(1)
        
        return (prev_bearish_marubozu & curr_bullish_marubozu & gap_up).fillna(False)
    
    def detect_kicking_bearish(self):
        """Detect Bearish Kicking pattern"""
        if len(self.data) < 2:
            return pd.Series([False] * len(self.data))
        
        prev_bullish_marubozu = self.data['IsGreen'].shift(1) & self.data['IsMarubozu'].shift(1)
        curr_bearish_marubozu = self.data['IsRed'] & self.data['IsMarubozu']
        gap_down = self.data['Open'] < self.data['Low'].shift(1)
        
        return (prev_bullish_marubozu & curr_bearish_marubozu & gap_down).fillna(False)
    
    def detect_belt_hold_bullish(self):
        """Detect Bullish Belt Hold pattern"""
        # Opens at the low and closes near the high with minimal lower shadow
        opens_at_low = self.data['Open'] <= self.data['Low'] + self.data['Range'] * 0.05
        closes_near_high = self.data['Close'] >= self.data['High'] - self.data['Range'] * 0.1
        is_green = self.data['IsGreen']
        
        return opens_at_low & closes_near_high & is_green
    
    def detect_belt_hold_bearish(self):
        """Detect Bearish Belt Hold pattern"""
        # Opens at the high and closes near the low with minimal upper shadow
        opens_at_high = self.data['Open'] >= self.data['High'] - self.data['Range'] * 0.05
        closes_near_low = self.data['Close'] <= self.data['Low'] + self.data['Range'] * 0.1
        is_red = self.data['IsRed']
        
        return opens_at_high & closes_near_low & is_red
    
    def detect_separating_lines_bullish(self):
        """Detect Bullish Separating Lines pattern"""
        if len(self.data) < 2:
            return pd.Series([False] * len(self.data))
        
        prev_red = self.data['IsRed'].shift(1)
        curr_green = self.data['IsGreen']
        same_opening = abs(self.data['Open'] - self.data['Open'].shift(1)) < self.data['Range'] * 0.02
        
        return (prev_red & curr_green & same_opening).fillna(False)
    
    def detect_separating_lines_bearish(self):
        """Detect Bearish Separating Lines pattern"""
        if len(self.data) < 2:
            return pd.Series([False] * len(self.data))
        
        prev_green = self.data['IsGreen'].shift(1)
        curr_red = self.data['IsRed']
        same_opening = abs(self.data['Open'] - self.data['Open'].shift(1)) < self.data['Range'] * 0.02
        
        return (prev_green & curr_red & same_opening).fillna(False)
    
    def detect_rising_three_methods(self):
        """Detect Rising Three Methods pattern (5-candle continuation)"""
        if len(self.data) < 5:
            return pd.Series([False] * len(self.data))
        
        # First candle: Large green
        first_green = self.data['IsGreen'].shift(4)
        first_large = self.data['Body'].shift(4) > self.data['Body'].shift(4).rolling(10).mean()
        
        # Next three candles: Small bodies, contained within first candle's range
        for i in range(3, 0, -1):
            small_body = self.data['Body'].shift(i) < self.data['Body'].shift(4) * 0.3
            contained = (
                (self.data['High'].shift(i) < self.data['High'].shift(4)) &
                (self.data['Low'].shift(i) > self.data['Low'].shift(4))
            )
            if i == 3:
                middle_conditions = small_body & contained
            else:
                middle_conditions = middle_conditions & small_body & contained
        
        # Fifth candle: Green, closes above first candle's high
        fifth_green = self.data['IsGreen']
        fifth_closes_high = self.data['Close'] > self.data['High'].shift(4)
        
        return (first_green & first_large & middle_conditions & fifth_green & fifth_closes_high).fillna(False)
    
    def detect_falling_three_methods(self):
        """Detect Falling Three Methods pattern (5-candle continuation)"""
        if len(self.data) < 5:
            return pd.Series([False] * len(self.data))
        
        # First candle: Large red
        first_red = self.data['IsRed'].shift(4)
        first_large = self.data['Body'].shift(4) > self.data['Body'].shift(4).rolling(10).mean()
        
        # Next three candles: Small bodies, contained within first candle's range
        for i in range(3, 0, -1):
            small_body = self.data['Body'].shift(i) < self.data['Body'].shift(4) * 0.3
            contained = (
                (self.data['High'].shift(i) < self.data['High'].shift(4)) &
                (self.data['Low'].shift(i) > self.data['Low'].shift(4))
            )
            if i == 3:
                middle_conditions = small_body & contained
            else:
                middle_conditions = middle_conditions & small_body & contained
        
        # Fifth candle: Red, closes below first candle's low
        fifth_red = self.data['IsRed']
        fifth_closes_low = self.data['Close'] < self.data['Low'].shift(4)
        
        return (first_red & first_large & middle_conditions & fifth_red & fifth_closes_low).fillna(False)
    
    def detect_mat_hold(self):
        """Detect Mat Hold pattern (5-candle bullish continuation)"""
        if len(self.data) < 5:
            return pd.Series([False] * len(self.data))
        
        # First candle: Green with gap up
        first_green = self.data['IsGreen'].shift(4)
        gap_up = self.data['Low'].shift(4) > self.data['High'].shift(5)
        
        # Next three candles: Small red bodies
        middle_red_1 = self.data['IsRed'].shift(3) & (self.data['Body'].shift(3) < self.data['Body'].shift(4) * 0.3)
        middle_red_2 = self.data['IsRed'].shift(2) & (self.data['Body'].shift(2) < self.data['Body'].shift(4) * 0.3)
        middle_red_3 = self.data['IsRed'].shift(1) & (self.data['Body'].shift(1) < self.data['Body'].shift(4) * 0.3)
        
        # Fifth candle: Green, closes above first candle's close
        fifth_green = self.data['IsGreen']
        fifth_closes_high = self.data['Close'] > self.data['Close'].shift(4)
        
        return (first_green & gap_up & middle_red_1 & middle_red_2 & middle_red_3 & fifth_green & fifth_closes_high).fillna(False)
    
    def detect_upside_tasuki_gap(self):
        """Detect Upside Tasuki Gap pattern"""
        if len(self.data) < 3:
            return pd.Series([False] * len(self.data))
        
        # First two candles: Both green with gap between them
        first_green = self.data['IsGreen'].shift(2)
        second_green = self.data['IsGreen'].shift(1)
        gap_between = self.data['Low'].shift(1) > self.data['High'].shift(2)
        
        # Third candle: Red, opens within second candle's body and closes within the gap
        third_red = self.data['IsRed']
        opens_in_second = (
            (self.data['Open'] > self.data['Open'].shift(1)) &
            (self.data['Open'] < self.data['Close'].shift(1))
        )
        closes_in_gap = (
            (self.data['Close'] > self.data['High'].shift(2)) &
            (self.data['Close'] < self.data['Low'].shift(1))
        )
        
        return (first_green & second_green & gap_between & third_red & opens_in_second & closes_in_gap).fillna(False)
    
    def detect_downside_tasuki_gap(self):
        """Detect Downside Tasuki Gap pattern"""
        if len(self.data) < 3:
            return pd.Series([False] * len(self.data))
        
        # First two candles: Both red with gap between them
        first_red = self.data['IsRed'].shift(2)
        second_red = self.data['IsRed'].shift(1)
        gap_between = self.data['High'].shift(1) < self.data['Low'].shift(2)
        
        # Third candle: Green, opens within second candle's body and closes within the gap
        third_green = self.data['IsGreen']
        opens_in_second = (
            (self.data['Open'] < self.data['Open'].shift(1)) &
            (self.data['Open'] > self.data['Close'].shift(1))
        )
        closes_in_gap = (
            (self.data['Close'] < self.data['Low'].shift(2)) &
            (self.data['Close'] > self.data['High'].shift(1))
        )
        
        return (first_red & second_red & gap_between & third_green & opens_in_second & closes_in_gap).fillna(False)
    
    def detect_shooting_star(self, threshold=2):
        """Detect Shooting Star pattern"""
        conditions = (
            (self.data['UpperShadow'] >= threshold * self.data['Body']) &
            (self.data['LowerShadow'] <= 0.1 * self.data['Body']) &
            (self.data['Body'] > 0)
        )
        return conditions
    
    def detect_morning_star(self):
        """Detect Morning Star pattern (3-candle pattern)"""
        if len(self.data) < 3:
            return pd.Series([False] * len(self.data))
        
        first_red = self.data['IsRed'].shift(2)
        second_small = (self.data['Body'].shift(1) < self.data['Body'].shift(2) * 0.3)
        third_green = self.data['IsGreen']
        third_closes_above_first = self.data['Close'] > (self.data['Open'].shift(2) + self.data['Close'].shift(2)) / 2
        
        return (first_red & second_small & third_green & third_closes_above_first).fillna(False)
    
    def detect_evening_star(self):
        """Detect Evening Star pattern (3-candle pattern)"""
        if len(self.data) < 3:
            return pd.Series([False] * len(self.data))
        
        first_green = self.data['IsGreen'].shift(2)
        second_small = (self.data['Body'].shift(1) < self.data['Body'].shift(2) * 0.3)
        third_red = self.data['IsRed']
        third_closes_below_first = self.data['Close'] < (self.data['Open'].shift(2) + self.data['Close'].shift(2)) / 2
        
        return (first_green & second_small & third_red & third_closes_below_first).fillna(False)
    
    def detect_all_patterns(self):
        """Detect all patterns and return results"""
        results = pd.DataFrame(index=self.data.index)
        
        # Original patterns
        results['Hammer'] = self.detect_hammer()
        results['Hanging_Man'] = self.detect_hanging_man()
        results['Doji'] = self.detect_doji()
        results['Bullish_Engulfing'] = self.detect_engulfing_bullish()
        results['Bearish_Engulfing'] = self.detect_engulfing_bearish()
        results['Shooting_Star'] = self.detect_shooting_star()
        results['Morning_Star'] = self.detect_morning_star()
        results['Evening_Star'] = self.detect_evening_star()
        
        # New patterns
        results['Three_Inside_Up'] = self.detect_three_inside_up()
        results['Three_Inside_Down'] = self.detect_three_inside_down()
        results['Rising_Three_Methods'] = self.detect_rising_three_methods()
        results['Falling_Three_Methods'] = self.detect_falling_three_methods()
        results['Bullish_Harami'] = self.detect_harami_bullish()
        results['Bearish_Harami'] = self.detect_harami_bearish()
        results['Bullish_Harami_Cross'] = self.detect_harami_cross_bullish()
        results['Bearish_Harami_Cross'] = self.detect_harami_cross_bearish()
        results['Kicking_Bullish'] = self.detect_kicking_bullish()
        results['Kicking_Bearish'] = self.detect_kicking_bearish()
        results['Long_Legged_Doji'] = self.detect_long_legged_doji()
        results['Marubozu_Bullish'] = self.detect_marubozu_bullish()
        results['Marubozu_Bearish'] = self.detect_marubozu_bearish()
        results['High_Wave_Candle'] = self.detect_high_wave_candle()
        results['Mat_Hold'] = self.detect_mat_hold()
        results['Upside_Tasuki_Gap'] = self.detect_upside_tasuki_gap()
        results['Downside_Tasuki_Gap'] = self.detect_downside_tasuki_gap()
        results['Bullish_Belt_Hold'] = self.detect_belt_hold_bullish()
        results['Bearish_Belt_Hold'] = self.detect_belt_hold_bearish()
        results['Separating_Lines_Bullish'] = self.detect_separating_lines_bullish()
        results['Separating_Lines_Bearish'] = self.detect_separating_lines_bearish()
        
        return results
    
    def calculate_pattern_win_rates(self, patterns=None, lookforward_periods=[1, 3, 5, 10]):
        """
        Calculate win rates for each pattern based on future price movements
        
        Parameters:
        patterns: DataFrame with pattern detection results
        lookforward_periods: List of periods to look forward for win rate calculation
        
        Returns:
        Dictionary with win rates for each pattern and lookforward period
        """
        if patterns is None:
            patterns = self.detect_all_patterns()
        
        # Define expected direction for each pattern - UPDATED with new patterns
        bullish_patterns = [
            'Hammer', 'Bullish_Engulfing', 'Morning_Star', 'Three_Inside_Up', 
            'Rising_Three_Methods', 'Bullish_Harami', 'Bullish_Harami_Cross',
            'Kicking_Bullish', 'Marubozu_Bullish', 'Mat_Hold', 'Upside_Tasuki_Gap',
            'Bullish_Belt_Hold', 'Separating_Lines_Bullish'
        ]
        bearish_patterns = [
            'Hanging_Man', 'Bearish_Engulfing', 'Shooting_Star', 'Evening_Star',
            'Three_Inside_Down', 'Falling_Three_Methods', 'Bearish_Harami',
            'Bearish_Harami_Cross', 'Kicking_Bearish', 'Marubozu_Bearish',
            'Downside_Tasuki_Gap', 'Bearish_Belt_Hold', 'Separating_Lines_Bearish'
        ]
        neutral_patterns = ['Doji', 'Long_Legged_Doji', 'High_Wave_Candle']
        
        win_rates = {}
        
        for pattern in patterns.columns:
            pattern_occurrences = patterns[patterns[pattern]].index
            if len(pattern_occurrences) == 0:
                continue
                
            win_rates[pattern] = {}
            
            for period in lookforward_periods:
                wins = 0
                total = 0
                
                for idx in pattern_occurrences:
                    # Get current index position
                    try:
                        current_pos = self.data.index.get_loc(idx)
                    except KeyError:
                        continue
                    
                    # Check if we have enough future data
                    if current_pos + period >= len(self.data):
                        continue
                    
                    current_close = self.data.iloc[current_pos]['Close']
                    future_close = self.data.iloc[current_pos + period]['Close']
                    
                    # Determine if it's a win based on pattern expectation
                    if pattern in bullish_patterns:
                        # Bullish pattern: win if future price is higher
                        win = future_close > current_close
                    elif pattern in bearish_patterns:
                        # Bearish pattern: win if future price is lower
                        win = future_close < current_close
                    else:  # neutral patterns
                        # For neutral patterns, consider significant moves in either direction
                        price_change_pct = abs((future_close - current_close) / current_close)
                        win = price_change_pct < 0.02  # Less than 2% change is considered "neutral success"
                    
                    if win:
                        wins += 1
                    total += 1
                
                win_rates[pattern][f'{period}_periods'] = {
                    'win_rate': wins / total if total > 0 else 0,
                    'total_signals': total,
                    'wins': wins
                }
        
        return win_rates
    
    def print_win_rate_summary(self, win_rates):
        """Print a formatted summary of win rates"""
        print("Pattern Win Rate Summary")
        print("=" * 60)
        
        for pattern, periods in win_rates.items():
            print(f"\n{pattern}:")
            for period, stats in periods.items():
                win_rate = stats['win_rate'] * 100
                total = stats['total_signals']
                wins = stats['wins']
                print(f"  {period}: {win_rate:.1f}% ({wins}/{total})")
    
    def plot_with_patterns(self, patterns=None, title="Candlestick Chart with Patterns", last_n=None):
        """
        Plot candlestick chart with detected patterns
        
        Parameters:
        patterns: DataFrame with pattern detection results
        title: Chart title
        last_n: If provided, plot only the last n data points
        """
        if patterns is None:
            patterns = self.detect_all_patterns()
        
        # Filter data to last n points if specified
        if last_n is not None:
            plot_data = self.data.tail(last_n).copy()
            plot_patterns = patterns.tail(last_n).copy()
        else:
            plot_data = self.data.copy()
            plot_patterns = patterns.copy()
        
        fig = go.Figure()
        
        # Add candlestick chart
        fig.add_trace(go.Candlestick(
            x=plot_data.index,
            open=plot_data['Open'],
            high=plot_data['High'],
            low=plot_data['Low'],
            close=plot_data['Close'],
            name="OHLC"
        ))
        
        # Add pattern markers - UPDATED with new patterns
        colors = {
            'Hammer': 'green', 'Hanging_Man': 'orange', 'Doji': 'blue',
            'Bullish_Engulfing': 'lime', 'Bearish_Engulfing': 'red',
            'Shooting_Star': 'purple', 'Morning_Star': 'cyan', 'Evening_Star': 'magenta',
            'Three_Inside_Up': 'lightgreen', 'Three_Inside_Down': 'lightcoral',
            'Rising_Three_Methods': 'forestgreen', 'Falling_Three_Methods': 'darkred',
            'Bullish_Harami': 'springgreen', 'Bearish_Harami': 'salmon',
            'Bullish_Harami_Cross': 'mediumspringgreen', 'Bearish_Harami_Cross': 'lightsalmon',
            'Kicking_Bullish': 'darkgreen', 'Kicking_Bearish': 'darkred',
            'Long_Legged_Doji': 'navy', 'Marubozu_Bullish': 'limegreen',
            'Marubozu_Bearish': 'crimson', 'High_Wave_Candle': 'royalblue',
            'Mat_Hold': 'seagreen', 'Upside_Tasuki_Gap': 'mediumseagreen',
            'Downside_Tasuki_Gap': 'indianred', 'Bullish_Belt_Hold': 'yellowgreen',
            'Bearish_Belt_Hold': 'firebrick', 'Separating_Lines_Bullish': 'lightseagreen',
            'Separating_Lines_Bearish': 'rosybrown'
        }
        
        for pattern, color in colors.items():
            if pattern in plot_patterns.columns:
                pattern_indices = plot_patterns[plot_patterns[pattern]].index
                if len(pattern_indices) > 0:
                    fig.add_trace(go.Scatter(
                        x=pattern_indices,
                        y=plot_data.loc[pattern_indices, 'High'] * 1.02,
                        mode='markers',
                        marker=dict(color=color, size=10, symbol='triangle-up'),
                        name=pattern,
                        text=pattern,
                        hovertemplate=f"{pattern}<extra></extra>"
                    ))
        
        fig.update_layout(
            title=title,
            xaxis_title="Date",
            yaxis_title="Price",
            xaxis_rangeslider_visible=False,
            height=600
        )
        
        return fig

# Example usage:
# Create sample data or load your own OHLC data
# df = pd.read_csv('your_ohlc_data.csv')  # Should have Open, High, Low, Close columns
# recognizer = CandlestickPatternRecognizer(df)
# patterns = recognizer.detect_all_patterns()
# fig = recognizer.plot_with_patterns(patterns)
# fig.show()

In [2]:
data = pd.read_csv('../data/BTC_chan_analysis.csv')
recognizer = CandlestickPatternRecognizer(data[['open', 'high', 'low', 'close']])
patterns = recognizer.detect_all_patterns()

# Calculate win rates for patterns
win_rates = recognizer.calculate_pattern_win_rates(patterns, lookforward_periods=[1, 3, 5, 10, 20])
recognizer.print_win_rate_summary(win_rates)

Pattern Win Rate Summary

Hammer:
  1_periods: 35.7% (5/14)
  3_periods: 64.3% (9/14)
  5_periods: 71.4% (10/14)
  10_periods: 64.3% (9/14)
  20_periods: 78.6% (11/14)

Hanging_Man:
  1_periods: 64.3% (9/14)
  3_periods: 35.7% (5/14)
  5_periods: 28.6% (4/14)
  10_periods: 35.7% (5/14)
  20_periods: 21.4% (3/14)

Doji:
  1_periods: 99.0% (196/198)
  3_periods: 39.9% (79/198)
  5_periods: 35.9% (71/198)
  10_periods: 28.3% (56/198)
  20_periods: 16.7% (33/198)

Bullish_Engulfing:
  1_periods: 0.0% (0/112)
  3_periods: 20.5% (23/112)
  5_periods: 29.5% (33/112)
  10_periods: 34.8% (39/112)
  20_periods: 38.4% (43/112)

Bearish_Engulfing:
  1_periods: 0.0% (0/130)
  3_periods: 17.7% (23/130)
  5_periods: 23.8% (31/130)
  10_periods: 36.2% (47/130)
  20_periods: 37.2% (48/129)

Shooting_Star:
  1_periods: 75.0% (3/4)
  3_periods: 25.0% (1/4)
  5_periods: 25.0% (1/4)
  10_periods: 75.0% (3/4)
  20_periods: 25.0% (1/4)

Morning_Star:
  1_periods: 0.9% (1/116)
  3_periods: 33.6% (39/116)
  5_

In [20]:
# Additional analysis options

# 1. Plot only the last 50 periods for a closer look
print("Plotting last 50 periods:")
fig_recent = recognizer.plot_with_patterns(patterns, title="BTC Recent Patterns (Last 50 periods)", last_n=50)
fig_recent.show()

# 2. Detailed win rate analysis with different time horizons
print("\nDetailed Win Rate Analysis:")
detailed_win_rates = recognizer.calculate_pattern_win_rates(patterns, lookforward_periods=[1, 2, 3, 5, 7, 10, 15, 20])

# 3. Show pattern statistics
print("\nPattern Occurrence Statistics:")
pattern_counts = patterns.sum()
print("Total pattern occurrences:")
for pattern, count in pattern_counts.items():
    if count > 0:
        print(f"  {pattern}: {count} times")

# 4. Show recent pattern occurrences (last 20 periods)
print("\nRecent Pattern Occurrences (Last 20 periods):")
recent_patterns = patterns.tail(20)
recent_data = data.tail(20)

for idx in recent_patterns.index[-20:]:
    row_patterns = recent_patterns.loc[idx]
    active_patterns = row_patterns[row_patterns == True].index.tolist()
    if active_patterns:
        date_info = f"Index {idx}" if not hasattr(data.index, 'date') else str(idx)
        print(f"  {date_info}: {', '.join(active_patterns)}")

Plotting last 50 periods:



Detailed Win Rate Analysis:

Pattern Occurrence Statistics:
Total pattern occurrences:
  Hammer: 14 times
  Hanging_Man: 14 times
  Doji: 198 times
  Bullish_Engulfing: 112 times
  Bearish_Engulfing: 131 times
  Shooting_Star: 4 times
  Morning_Star: 117 times
  Evening_Star: 107 times
  Three_Inside_Up: 54 times
  Three_Inside_Down: 50 times
  Bullish_Harami: 109 times
  Bearish_Harami: 85 times
  Bullish_Harami_Cross: 13 times
  Bearish_Harami_Cross: 19 times
  Long_Legged_Doji: 198 times
  Marubozu_Bullish: 12 times
  Marubozu_Bearish: 25 times
  High_Wave_Candle: 495 times
  Bullish_Belt_Hold: 22 times
  Bearish_Belt_Hold: 36 times
  Separating_Lines_Bullish: 9 times
  Separating_Lines_Bearish: 7 times

Recent Pattern Occurrences (Last 20 periods):
  Index 1807: Bearish_Harami
  Index 1808: Evening_Star, Three_Inside_Down
  Index 1809: Doji, Bullish_Harami, Long_Legged_Doji, High_Wave_Candle
  Index 1810: Morning_Star, Three_Inside_Up
  Index 1818: Bearish_Harami
  Index 1819: Bea

In [11]:
# Test the new patterns
print("Testing new pattern detection...")
print(f"Total patterns being detected: {len(patterns.columns)}")
print("\nAll pattern columns:")
for i, col in enumerate(patterns.columns, 1):
    count = patterns[col].sum()
    print(f"{i:2d}. {col}: {count} occurrences")

print(f"\nNew patterns summary:")
new_patterns = ['Three_Inside_Up', 'Three_Inside_Down', 'Rising_Three_Methods', 'Falling_Three_Methods',
               'Bullish_Harami', 'Bearish_Harami', 'Bullish_Harami_Cross', 'Bearish_Harami_Cross',
               'Kicking_Bullish', 'Kicking_Bearish', 'Long_Legged_Doji', 'Marubozu_Bullish', 
               'Marubozu_Bearish', 'High_Wave_Candle', 'Mat_Hold', 'Upside_Tasuki_Gap', 
               'Downside_Tasuki_Gap', 'Bullish_Belt_Hold', 'Bearish_Belt_Hold', 
               'Separating_Lines_Bullish', 'Separating_Lines_Bearish']

new_pattern_counts = 0
for pattern in new_patterns:
    if pattern in patterns.columns:
        count = patterns[pattern].sum()
        new_pattern_counts += count
        if count > 0:
            print(f"  {pattern}: {count} times")

print(f"\nTotal new pattern detections: {new_pattern_counts}")

# Let's check some recent data for specific pattern types
print(f"\nChecking recent 10 rows for pattern detection:")
recent_data = patterns.tail(10)
recent_info = data.tail(10)

for idx in range(len(recent_data)):
    row = recent_data.iloc[idx]
    active = row[row == True].index.tolist()
    if active:
        date_idx = recent_info.iloc[idx].name if hasattr(recent_info.iloc[idx], 'name') else f"Row {idx}"
        print(f"  {date_idx}: {', '.join(active)}")

Testing new pattern detection...
Total patterns being detected: 8

All pattern columns:
 1. Hammer: 14 occurrences
 2. Hanging_Man: 14 occurrences
 3. Doji: 198 occurrences
 4. Bullish_Engulfing: 112 occurrences
 5. Bearish_Engulfing: 131 occurrences
 6. Shooting_Star: 4 occurrences
 7. Morning_Star: 117 occurrences
 8. Evening_Star: 107 occurrences

New patterns summary:

Total new pattern detections: 0

Checking recent 10 rows for pattern detection:
  1819: Bearish_Engulfing
  1824: Morning_Star


In [21]:
# Calculate win rates for all patterns and run trading analysis
print("Calculating win rates for all patterns...")
detailed_win_rates = recognizer.calculate_pattern_win_rates(patterns, lookforward_periods=[1, 2, 3, 5, 7, 10, 15, 20])

# Show recent trading analysis with all new patterns
latest_date = data.iloc[-1]['date']
latest_price = data.iloc[-1]['close']
latest_patterns = patterns.iloc[-1]

print(f"\n=== BTC Trading Analysis for {latest_date} ===")
print(f"Current Price: ${latest_price:,.2f}")
print()

# Check for active patterns in the most recent candle
active_patterns = latest_patterns[latest_patterns == True].index.tolist()

if active_patterns:
    print("🔍 ACTIVE PATTERNS DETECTED:")
    for pattern in active_patterns:
        print(f"  • {pattern}")
    print()
    
    # Categorize patterns - UPDATED with new patterns
    bullish_patterns = [
        'Hammer', 'Bullish_Engulfing', 'Morning_Star', 'Three_Inside_Up', 
        'Rising_Three_Methods', 'Bullish_Harami', 'Bullish_Harami_Cross',
        'Kicking_Bullish', 'Marubozu_Bullish', 'Mat_Hold', 'Upside_Tasuki_Gap',
        'Bullish_Belt_Hold', 'Separating_Lines_Bullish'
    ]
    bearish_patterns = [
        'Hanging_Man', 'Bearish_Engulfing', 'Shooting_Star', 'Evening_Star',
        'Three_Inside_Down', 'Falling_Three_Methods', 'Bearish_Harami',
        'Bearish_Harami_Cross', 'Kicking_Bearish', 'Marubozu_Bearish',
        'Downside_Tasuki_Gap', 'Bearish_Belt_Hold', 'Separating_Lines_Bearish'
    ]
    neutral_patterns = ['Doji', 'Long_Legged_Doji', 'High_Wave_Candle']
    
    bullish_signals = [p for p in active_patterns if p in bullish_patterns]
    bearish_signals = [p for p in active_patterns if p in bearish_patterns]
    neutral_signals = [p for p in active_patterns if p in neutral_patterns]
    
    print("📊 TRADING RECOMMENDATIONS:")
    
    if bullish_signals:
        print("🟢 BULLISH BIAS:")
        for pattern in bullish_signals:
            if pattern in detailed_win_rates:
                win_rate_1d = detailed_win_rates[pattern]['1_periods']['win_rate'] * 100
                win_rate_5d = detailed_win_rates[pattern]['5_periods']['win_rate'] * 100
                total_signals = detailed_win_rates[pattern]['1_periods']['total_signals']
                print(f"  • {pattern}: {win_rate_1d:.1f}% win rate (1 day), {win_rate_5d:.1f}% (5 days) | {total_signals} historical signals")
            else:
                print(f"  • {pattern}: New pattern - insufficient historical data")
        print("  → Consider LONG positions with proper risk management")
        print(f"  → Stop loss: Below ${latest_price * 0.97:.2f} (-3%)")
        print(f"  → Take profit: Above ${latest_price * 1.05:.2f} (+5%)")
    
    if bearish_signals:
        print("🔴 BEARISH BIAS:")
        for pattern in bearish_signals:
            if pattern in detailed_win_rates:
                win_rate_1d = detailed_win_rates[pattern]['1_periods']['win_rate'] * 100
                win_rate_5d = detailed_win_rates[pattern]['5_periods']['win_rate'] * 100
                total_signals = detailed_win_rates[pattern]['1_periods']['total_signals']
                print(f"  • {pattern}: {win_rate_1d:.1f}% win rate (1 day), {win_rate_5d:.1f}% (5 days) | {total_signals} historical signals")
            else:
                print(f"  • {pattern}: New pattern - insufficient historical data")
        print("  → Consider SHORT positions or profit-taking")
        print(f"  → Stop loss: Above ${latest_price * 1.03:.2f} (+3%)")
        print(f"  → Take profit: Below ${latest_price * 0.95:.2f} (-5%)")
    
    if neutral_signals:
        print("⚪ NEUTRAL/INDECISION:")
        for pattern in neutral_signals:
            if pattern in detailed_win_rates:
                win_rate_1d = detailed_win_rates[pattern]['1_periods']['win_rate'] * 100
                total_signals = detailed_win_rates[pattern]['1_periods']['total_signals']
                print(f"  • {pattern}: {win_rate_1d:.1f}% success rate | {total_signals} historical signals")
            else:
                print(f"  • {pattern}: New pattern - insufficient historical data")
        print("  → Wait for confirmation before entering positions")

    # Check for conflicting signals
    if bullish_signals and bearish_signals:
        print("\n⚠️  WARNING: Conflicting signals detected!")
        print("   → Exercise extra caution and wait for clearer direction")

else:
    print("🔍 NO ACTIVE PATTERNS DETECTED")
    print("📊 Wait for pattern formation before major trades")

print(f"\n✅ Analysis complete with {len(patterns.columns)} pattern types!")
print(f"📈 New patterns successfully added to the system.")

Calculating win rates for all patterns...

=== BTC Trading Analysis for 2025-10-30 ===
Current Price: $110,066.00

🔍 ACTIVE PATTERNS DETECTED:
  • Bearish_Harami

📊 TRADING RECOMMENDATIONS:
🔴 BEARISH BIAS:
  • Bearish_Harami: 52.4% win rate (1 day), 54.8% (5 days) | 84 historical signals
  → Consider SHORT positions or profit-taking
  → Stop loss: Above $113367.98 (+3%)
  → Take profit: Below $104562.70 (-5%)

✅ Analysis complete with 29 pattern types!
📈 New patterns successfully added to the system.

=== BTC Trading Analysis for 2025-10-30 ===
Current Price: $110,066.00

🔍 ACTIVE PATTERNS DETECTED:
  • Bearish_Harami

📊 TRADING RECOMMENDATIONS:
🔴 BEARISH BIAS:
  • Bearish_Harami: 52.4% win rate (1 day), 54.8% (5 days) | 84 historical signals
  → Consider SHORT positions or profit-taking
  → Stop loss: Above $113367.98 (+3%)
  → Take profit: Below $104562.70 (-5%)

✅ Analysis complete with 29 pattern types!
📈 New patterns successfully added to the system.
